In [1]:
from researcher import Researcher
import asyncio
import nest_asyncio

nest_asyncio.apply()

In [2]:
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8042
WEBHOOK_BASE = ""
ADMIN_URL = "http://researcher-agent:8041"
# ADMIN_URL = "http://0.0.0.0:8041"

agent_config = {
    "webhook_host": WEBHOOK_HOST,
    "webhook_port": WEBHOOK_PORT,
    "webhook_base": WEBHOOK_BASE,
    "admin_url": ADMIN_URL
}



In [3]:
import torch

# models
from torch import nn
from torch import optim
from torch.autograd import Variable
import os
import sys

from opacus import PrivacyEngine

# The Researcher generates the model

model = nn.Sequential(
            nn.Linear(8, 4),
            nn.Sigmoid(),
            nn.Linear(4, 2),
            nn.Sigmoid(),
            nn.Linear(2, 1),
            nn.Sigmoid()
)

In [4]:
validation_data_path = 'data/coordinator_validation_data.csv'

In [5]:
researcher = Researcher(agent_config, validation_data_path, model)

CLEANING THE VALIDATION SET
VALIDATION SET HAS BEEN CLEANED


## Request Authorised Research Credential

Copy invite from the [Health Research Authority Notebook](http://localhost:8889/lab/tree/Health%20Research%20Regulator.ipynb)

In [6]:
invite = {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '6cccd38d-b34b-4798-aa7c-b529ddd315e0', 'recipientKeys': ['Fm7bvrNNTJCuT1RKjPqcNg96Xcamm6mwivNgQ7CpU8ph'], 'serviceEndpoint': 'http://172.17.0.1:8030', 'label': 'NHS Health Research Authority'}

In [7]:
await researcher.agent_controller.connections.accept_connection(invite)

Connection Handler Called
Connection 6359cf14-0456-4662-89ef-b3910e25b0aa in State invitation
Connection Handler Called
Connection 6359cf14-0456-4662-89ef-b3910e25b0aa in State request


{'state': 'request',
 'connection_id': '6359cf14-0456-4662-89ef-b3910e25b0aa',
 'updated_at': '2021-03-01 19:57:54.019977Z',
 'my_did': 'MbmxaqUNq4TXBPHyk3QQ1x',
 'their_label': 'NHS Health Research Authority',
 'request_id': '43c376c1-8b05-4b1c-a163-03d9966a1c60',
 'invitation_mode': 'once',
 'initiator': 'external',
 'accept': 'manual',
 'invitation_key': 'Fm7bvrNNTJCuT1RKjPqcNg96Xcamm6mwivNgQ7CpU8ph',
 'routing_state': 'none',
 'created_at': '2021-03-01 19:57:54.006166Z'}

Connection Handler Called
Connection 6359cf14-0456-4662-89ef-b3910e25b0aa in State response
Connection Handler Called
Connection 6359cf14-0456-4662-89ef-b3910e25b0aa in State active
Handle Credentials
Credential exchange f6fb44c2-63a6-429c-8ab7-8660a5e200f6, role: holder, state: offer_received
Offering: [{'name': 'scope', 'value': 'diabetes'}]
Requesting credential
Handle Credentials
Credential exchange f6fb44c2-63a6-429c-8ab7-8660a5e200f6, role: holder, state: request_sent
Handle Credentials
Credential exchange f6fb44c2-63a6-429c-8ab7-8660a5e200f6, role: holder, state: credential_received
Storing Credential
Handle Credentials
Credential exchange f6fb44c2-63a6-429c-8ab7-8660a5e200f6, role: holder, state: credential_acked


## Configure Authentication Policy

Note: Normally we would specify the credential definition here specifying which entities we trust to issue this schema, but this changes every time because it's a demo.

Feel free to copy yours across from the NHS Trust though.

In [8]:


schema_id = "BvRKFu1fdzGdJzQcKb8e8p:2:NHS Hosptial:0.0.1"
# cred_def = "WfntKNFwXMQfgmU9ofbxPM:3:CL:156569:default"

revocation = False
exchange_tracing = False

#Enable this to ask for attributes to identity a user
#TODO - change restriction to schemaId or credentialId

req_attrs = [
    {"name": "name", "restrictions": [{"schema_id": schema_id}]},#, "cred_def_id": cred_def}]},
]

global indy_proof_request
indy_proof_request = {
    "name": "Proof of Hospital",
    "version": "1.0",
    "requested_attributes": {
        f"0_{req_attr['name']}_uuid":
        req_attr for req_attr in req_attrs
    },
    "requested_predicates": {
#         f"0_{req_pred['name']}_GE_uuid":
#         req_pred for req_pred in req_preds
    },
}



In [9]:
researcher.set_authentication_policy(indy_proof_request)

## Send Invite to Hospital 1

Option: We could make this a multiuse invite.

In [10]:
researcher.new_dataowner_invite()

Connection Handler Called
Connection 60b0df3b-b1e7-4c1a-8d77-82c18cd9a9eb in State invitation

♫♫♫ > STEP 1: Send the aries invitation to the data owner

{"@type": "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation", "@id": "2b58ba18-b610-41e9-9cd1-40218ce172cf", "recipientKeys": ["7R6SvZLAfV8qKGMQ2CrcNYXjGRexcnaKfaVwbu82gteN"], "serviceEndpoint": "http://172.17.0.1:8040", "label": "Health Researcher"}

Connection Handler Called
Connection 60b0df3b-b1e7-4c1a-8d77-82c18cd9a9eb in State request
Accepting request  60b0df3b-b1e7-4c1a-8d77-82c18cd9a9eb
Connection Handler Called
Connection 60b0df3b-b1e7-4c1a-8d77-82c18cd9a9eb in State response
Sending trust ping 60b0df3b-b1e7-4c1a-8d77-82c18cd9a9eb
Connection Handler Called
Connection 60b0df3b-b1e7-4c1a-8d77-82c18cd9a9eb in State active
Pending dataowner connection moved to active. Challenging with auth policy if set
Handle present proof
Role verifier, Exchange 50034b39-08cc-456e-8199-5505e9fcdfe9 in state request_sent
Handle pr

## Send Invite to Hospital 2

In [13]:
researcher.new_dataowner_invite()

Connection Handler Called
Connection 2fdd0e61-0b94-428d-bff7-f18f5dac8230 in State invitation

♫♫♫ > STEP 1: Send the aries invitation to the data owner

{"@type": "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation", "@id": "af4ca1f7-7070-42cc-943d-e7af24bef5c4", "label": "Health Researcher", "recipientKeys": ["DLbK5vsixEqenNevKHRxgK3jMZHApHHQZ12ix5FsM15Y"], "serviceEndpoint": "http://172.17.0.1:8040"}

Connection Handler Called
Connection 2fdd0e61-0b94-428d-bff7-f18f5dac8230 in State request
Accepting request  2fdd0e61-0b94-428d-bff7-f18f5dac8230
Connection Handler Called
Connection 2fdd0e61-0b94-428d-bff7-f18f5dac8230 in State response
Sending trust ping 2fdd0e61-0b94-428d-bff7-f18f5dac8230
Connection Handler Called
Connection 2fdd0e61-0b94-428d-bff7-f18f5dac8230 in State active
Pending dataowner connection moved to active. Challenging with auth policy if set
Handle present proof
Role verifier, Exchange 464cb34f-1daa-4107-a9d2-9ed7af092919 in state request_sent
Handle pr

## Send Invite to Hospital 3

In [ ]:
researcher.new_dataowner_invite()

## Initiate Learning

PPML Learning messages are sent over DIDComm using the basic message protocol. You will be able to see the encrypted message in the terminal you initialised this project.

In [11]:
researcher.initiate_learning()

MODEL OPENED FOR TRANSPORT
Training model with 1 DataOwners
HOSPITAL MODEL LOADED

PRINTING PARAMETERS:


Sequential(
  (0): Linear(in_features=8, out_features=4, bias=True)
  (1): Sigmoid()
  (2): Linear(in_features=4, out_features=2, bias=True)
  (3): Sigmoid()
  (4): Linear(in_features=2, out_features=1, bias=True)
  (5): Sigmoid()
)


0.weight tensor([[-0.2710, -0.9557,  0.2309, -1.9110, -0.9440, -0.4835,  1.8869,  3.2344],
        [-0.7837, -0.9277,  1.5270, -2.2951, -0.0703,  2.0951, -0.2947,  0.4189],
        [ 0.3087,  0.1126, -0.6512,  0.4391, -0.7844, -0.3249, -0.5029,  3.2722],
        [-2.2386,  1.0693, -1.0953, -1.6919, -0.6925,  0.4706,  0.1246,  0.6962]])
0.bias tensor([-1.0267, -0.2995, -2.2666,  1.8172])
2.weight tensor([[ 1.7648,  1.0091,  1.9958, -2.0196],
        [-2.6523, -2.2176, -2.9162,  4.3020]])
2.bias tensor([-1.2612,  2.2130])
4.weight tensor([[ 4.0461, -6.9314]])
4.bias tensor([0.5980])



HOSPITAL IS VALIDATING
Model loss on validation set:  tensor(-0.7815